<a href="https://colab.research.google.com/github/Haverholme/rf-vision/blob/master/foodcats/FoodCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install unidecode

    100% |████████████████████████████████| 245kB 16.9MB/s 


In [0]:
from contextlib import suppress
import unidecode
import pandas as pd
from google.colab import files

file = files.upload()
train = pd.read_stata('Set.dta')

Saving Set.dta to Set.dta


In [0]:
for k in range(train.shape[0]) :
    with suppress(Exception) :
        text = train.loc[k,'group'] + train.loc[k,'item']
        text = text.replace(' ', '')
        text = text.lower()
        text = unidecode.unidecode(text)
        text2 = ''
        for i in range(len(text)-2) :
            text2 = text2+' '+text[i:i+3]
        for i in range(len(text) - 3) :
            text2 = text2 + ' ' + text[i:i+4]
        for i in range(len(text) - 4) :
            text2 = text2 + ' ' + text[i:i+5]
        for i in range(len(text) - 5) :
            text2 = text2 + ' ' + text[i:i+6]
        train.loc[k,'bag'] = text2

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV #train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from joblib import dump
import numpy as np

gbc = GradientBoostingClassifier()
results = {}
param_grid = [
    {
         'n_estimators': list(np.arange(100,1001,100)),
         'subsample': [np.round(x/10, 1) for x in np.arange(1, 11)],
         'max_depth': list(np.arange(1, 11))
    }
]

grid = GridSearchCV(gbc, cv = 5, n_jobs = -1, param_grid = param_grid, verbose = 1)

In [0]:
bags = {}
bags['base'] = train.bag
for cat in ['a', 'd', 'f'] :
    train2 = train[train['cat1'] == cat]
    bags[cat] = train2.bag

In [0]:
for key in ['base', 'a', 'd', 'f'] :
    vec = CountVectorizer().fit_transform(bags[key])
    if key == 'base' :
        grid.fit(vec, train['cat1'])
    else :
        grid.fit(vec, train[train['cat1'] == key]['cat2'])
        
    results[key]['params'] = grid.best_params_
    results[key]['score'] = grid.best_score_
        
    pl = Pipeline([
        ('vec2', CountVectorizer()),
        ('gbc2', GradientBoostingClassifier(max_depth = grid.best_params_['max_depth'], \
                                       n_estimators = grid.best_params_['n_estimators'], \
                                       subsample = grid.best_params_['subsample'], \
                                       random_state = 4))
    ])
       
    if key == 'base' :
        pl.fit(bags[key], train['cat1'])
    else :
        pl.fit(bags[key], train[train['cat1'] == key]['cat2'])
        
    dump(pl, 'CatPL_' + key + '_3-6.joblib')

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 11.8min
